In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [15]:
# Read Womens Clothing E-Commerce Reviews.csv:
import pandas as pd
df = pd.read_csv('./Womens Clothing E-Commerce Reviews.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               23486 non-null  int64 
 1   Clothing ID              23486 non-null  int64 
 2   Age                      23486 non-null  int64 
 3   Title                    19676 non-null  object
 4   Review Text              22641 non-null  object
 5   Rating                   23486 non-null  int64 
 6   Recommended IND          23486 non-null  int64 
 7   Positive Feedback Count  23486 non-null  int64 
 8   Division Name            23472 non-null  object
 9   Department Name          23472 non-null  object
 10  Class Name               23472 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


In [16]:
df.head()


,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [17]:
# Only take the review and recommendation column
df = df.drop(df.columns[[0, 1, 2, 3, 5, 7, 8, 9, 10]], axis=1)
df = df.dropna()
df

,Review Text,Recommended IND
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1
...,...,...
23481,I was very happy to snag this dress at such a ...,1
23482,"It reminds me of maternity clothes. soft, stre...",1
23483,"This fit well, but the top was very see throug...",0
23484,I bought this dress for a wedding i have this ...,1


Review Text: reviews of women's clothes. Recommended Id: 1 means Recommended, 0 means Not Recommended

In [18]:
df.head()


,Review Text,Recommended IND
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1


In [19]:
# Split data into train data and test data
from sklearn.model_selection import train_test_split
review = df['Review Text'].values
recommendation = df['Recommended IND'].values
review_train, review_test, recommendation_train, recommendation_test = train_test_split(review, recommendation, test_size=0.25)

In [20]:
review_test.shape

(5661,)

In [21]:
# Tokenization of review texts
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=50000, oov_token='x')
tokenizer.fit_on_texts(review_train) 
tokenizer.fit_on_texts(review_test)
 
sequence_train = tokenizer.texts_to_sequences(review_train)
sequence_test = tokenizer.texts_to_sequences(review_test)

padded_train = pad_sequences(sequence_train) 
padded_test = pad_sequences(sequence_test)

In [22]:
# Neural network model with a sequential model
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=50000, output_dim=128),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# Training process
# In addition, if we want to display accuracy in each model training process we can add a metrics parameter and fill it with the string 'accuracy'.
num_epochs = 5
history = model.fit(padded_train, recommendation_train, epochs=num_epochs, 
                    validation_data=(padded_test, recommendation_test), verbose=2)

Epoch 1/5
531/531 - 25s - loss: 0.3138 - accuracy: 0.8654 - val_loss: 0.2548 - val_accuracy: 0.8928 - 25s/epoch - 48ms/step
Epoch 2/5
531/531 - 25s - loss: 0.2039 - accuracy: 0.9170 - val_loss: 0.2705 - val_accuracy: 0.8914 - 25s/epoch - 47ms/step
Epoch 3/5
531/531 - 25s - loss: 0.1592 - accuracy: 0.9370 - val_loss: 0.2945 - val_accuracy: 0.8884 - 25s/epoch - 47ms/step
Epoch 4/5
531/531 - 25s - loss: 0.1162 - accuracy: 0.9539 - val_loss: 0.3306 - val_accuracy: 0.8741 - 25s/epoch - 47ms/step
Epoch 5/5


In [ ]:
# Plotting training accuracy and validation accuracy
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], 'r', label='Accuracy Training')
plt.plot(history.history['val_accuracy'], 'b', label='Accuracy Validation')
plt.title('Accuracy Training and Validation')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc=0)
plt.show()

In [ ]:
# Plotting training loss and validation loss
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], 'r', label='Loss Training')
plt.plot(history.history['val_loss'], 'b', label='Loss Validation')
plt.title('Loss Training and Validation')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc=0)
plt.show()

You can tinker with the Sequential Model part to get great results like adding the dropout() function.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize and pad your review
my_review = ["dress is not good"]
result = tokenizer.texts_to_sequences(my_review)
result = pad_sequences(result)

# Make predictions
predictions = model.predict(result)
predicted_class = np.argmax(predictions, axis=-1)

# Print recommendation
if predicted_class == 0:
    print('Not Recommended')
elif predicted_class == 1:
    print('Recommended')


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming tokenizer and model are already loaded
my_review = ["dress is good"]
result = tokenizer.texts_to_sequences(my_review)
result = pad_sequences(result)

# Make predictions
predictions = model.predict(result)
predicted_class = np.argmax(predictions, axis=-1)

# Print recommendation
if predicted_class == 0:
    print('Not Recommended')
elif predicted_class == 1:
    print('Recommended')
